In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import models
import tensorflow as tf
import numpy as np
import glob

In [12]:
tf.keras.backend.clear_session()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9145533444723251860
]


In [13]:
def read_many(list_files):
    list_imgs=list(glob.glob(list_files))
    imgs=[]
    for i in range(len(list_imgs)):
        try:
            img = image.load_img(list_imgs[i], target_size=(224,224,3))
            x = image.img_to_array(img)
            imgs.append(x)
        except ValueError:
            print("error reading the following image:",list_imgs[i])
    return imgs


def load_dir(directory):
    
    covid=read_many("{}/COVID/*".format(directory))
    y_covid=[1]*len(covid)
    non_covid=read_many("{}/COVID/*".format(directory))
    y_non_covid=[0]*len(non_covid)
    X=np.concatenate([np.asarray(covid), np.asarray(non_covid)])
    Y=np.concatenate([np.asarray(y_covid), np.asarray(y_non_covid)])
    return (X, Y) 
    

In [14]:
X,Y=load_dir("Dataset/Train")

In [15]:
def CNN():
    conv_base = ResNet50(weights='imagenet',
                         #include_top = False,
                         #input_shape = (224,224, 3)
                        )


    # conv_base.summary()
    
    conv_base.trainable = True
    #model=conv_base
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation = 'relu'))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    
    #model.summary()
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])  
    
    return model

In [16]:
model=CNN()

In [9]:
model.fit(x=X/255,y=Y,epochs=10)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.6937 - accuracy: 0.3571
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 0.6956 - accuracy: 0.4286
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.6890 - accuracy: 0.6429
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.6836 - accuracy: 0.6429
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 0.6921 - accuracy: 0.4286
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 0.6926 - accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 0.6815 - accuracy: 0.6429
Epoch 8/10
1/1 [==============================] - 3s 3s/step - loss: 0.6943 - accuracy: 0.5000
Epoch 9/10
1/1 [==============================] - 4s 4s/step - loss: 0.6994 - accuracy: 0.4286
Epoch 10/10
1/1 [==============================] - 4s 4s/step - loss: 0.7005 - accuracy: 0.4286


In [111]:
model.predict(X/255)

array([[0.49341002, 0.5082426 ],
       [0.493464  , 0.5083346 ],
       [0.49345556, 0.5083247 ],
       [0.4934366 , 0.5083368 ],
       [0.49342066, 0.5084699 ],
       [0.4934574 , 0.5083052 ],
       [0.4934563 , 0.5083396 ],
       [0.49341002, 0.5082426 ],
       [0.493464  , 0.5083346 ],
       [0.49345556, 0.5083247 ],
       [0.4934366 , 0.5083368 ],
       [0.49342066, 0.5084699 ],
       [0.4934574 , 0.5083052 ],
       [0.4934563 , 0.5083396 ]], dtype=float32)

(14,)